# 2.

In [1]:
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [2]:
import os
import numpy as np
def load_NORB_train_val(PATH):
    xtr = []
    ytr = []
    xt  = []
    yt  = []
    xval = []
    yval = []
    for b in range(1,11):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        datadict = unpickle(f)
        X = np.array(datadict['data'].T).astype(float)
        Y = np.array(datadict['labels'],ndmin = 2).T
        Z = np.concatenate((X,Y),axis=1)
        np.random.shuffle(Z)
        xtr.append(Z[5832:,0:-1])
        ytr.append(Z[5832:,-1])
        xval.append(Z[:5832,0:-1])
        yval.append(Z[:5832,-1])
    Xtr = np.concatenate(xtr)
    Ytr = np.concatenate(ytr)
    Xval = np.concatenate(xval)
    Yval = np.concatenate(yval)
    del xtr,ytr,xval,yval
    for b in range(11,13):
        f = os.path.join(PATH, 'data_batch_%d' % (b, ))
        datadict = unpickle(f)
        X = np.array(datadict['data'].T).astype(float)
        Y = np.array(datadict['labels'],ndmin = 2).T
        Z = np.concatenate((X,Y),axis=1)
        xt.append(Z[:,0:-1])
        yt.append(Z[:,-1])
    Xt  = np.concatenate(xt)
    Yt  = np.concatenate(yt)
    del xt,yt
    return Xtr, Ytr, Xt, Yt, Xval, Yval

Xtr, Ytr, Xt, Yt, Xval, Yval = load_NORB_train_val(os.getcwd())

In [3]:
def get_scale_params(Xtr,Xt,Xval):
    Xmin = min(np.amin(Xtr),np.amin(Xt),np.amin(Xval))
    Xmax = max(np.amax(Xtr),np.amax(Xt),np.amax(Xval))
    Xrange = (Xmax-Xmin)/2.0
    Xcenter = (Xmax + Xmin)/2.0
    return Xrange,Xcenter

Xrange,Xcenter = get_scale_params(Xtr,Xt,Xval)

In [4]:
print(Xrange,Xcenter)

(127.5, 127.5)


In [5]:
def scale_pix(X,Xrange,Xcenter):
    X-=Xcenter
    X/=Xrange

scale_pix(Xtr ,Xrange,Xcenter)
scale_pix(Xt  ,Xrange,Xcenter)
scale_pix(Xval,Xrange,Xcenter)

In [13]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(Ytr, 6)
Y_test  = np_utils.to_categorical(Yt, 6)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
import time

def learn(Xtr,Ytr,Xt,Yt,theta):
    n_tr = len(Xtr)
    n_s  = int(n_tr*theta)
    X_train = Xtr[0:n_s,:]
    Y_train = Ytr[0:n_s,:]
    model = Sequential()
    model.add(Dense(4000, activation='relu',  input_shape=(2048,)))
    model.add(Dense(2000, activation='relu'))
    model.add(Dense(6, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    start = time.time()
    h = model.fit(X_train, Y_train, batch_size=10, nb_epoch=6, verbose=1, validation_data=(Xt, Yt))
    stop  = time.time()
    t = stop - start
    acc = h.history['val_acc'][-1]
    return acc,model,t,h

In [ ]:
import matplotlib.pyplot as plt
thetas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
acc = np.zeros(10)
for i in range(10):
    acc[i],model,t,h = learn(Xtr,Y_train,Xt,Y_test,thetas[i])
    model.save('q2c_theta'+str(thetas[i])+'.h5')
plt.plot(thetas,acc)
plt.title('model accuracy for different ')
plt.ylabel('accuracy reached')
plt.xlabel('theta (ratio of labelled data)')
plt.show()

WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Train on 23328 samples, validate on 58320 samples
Epoch 1/6
 7880/23328 [=========>....................] - ETA: 166s - loss: 0.4106 - acc: 0.8371